# Use Human-in-the-Loop to Improve Your Data Science Copilot Results

### Free Generative AI Data Science Workshop

If you want to learn how to build AI Agents that perform Data Science, Business Intelligence, Churn Modeling, Time Series Forecasting, and more, [register for my next free AI for Data Scientists workshop here.](https://learn.business-science.io/ai-register)


### What is Human-in-the-Loop?

Human-in-the-Loop is a concept where humans are involved in the decision-making process of AI models. The idea is that humans can provide feedback to AI models to improve their performance. This is especially useful in the case of AI models that are not perfect and need human intervention to improve their performance.

### What is Data Science Copilot (and how does Human-in-the-Loop Help)?

Data Science Copilot is a tool that helps data scientists write code faster and more efficiently. It is an AI-powered tool that provides code suggestions and auto-completion for data science tasks. Data Science Copilot can help data scientists complete common data science tasks faster.

**Human-in-the-Loop gives Data Scientists an opportunity to review the AI's plan before it executes code** to process the data or create machine learning models. This is important because the AI may not always provide the best suggestions, and human intervention can help improve the results.


## Example: Using Human-in-the-Loop to Improve Data Science Copilot Results for Feature Engineering

In this example, we will use Human-in-the-Loop to improve the results of Data Science Copilot for feature engineering. Feature engineering is an important step in the data science process, where we create new features and process existing features in data to improve the performance of machine learning models.

### Load Libraries

In [1]:
from langchain_openai import ChatOpenAI
import os
import yaml
import pandas as pd
from pprint import pprint

from ai_data_science_team.agents import make_data_cleaning_agent, make_feature_engineering_agent

from langgraph.types import Command

### Setup AI and Logging

This section of code sets up the LLM inputs and the logging information. Logging is used to store AI-generated code and files during the AI Data Science Teams processing of files. 

*Important Note:* This example uses OpenAI's API. But any LLM can be used such as Anthropic or local LLMs with Ollama.

In [2]:
# * Setup

MODEL    = "gpt-4o-mini"
LOG      = True
LOG_PATH = os.path.join(os.getcwd(), "logs/")

os.environ["OPENAI_API_KEY"] = yaml.safe_load(open('../credentials.yml'))['openai']

llm = ChatOpenAI(model = MODEL)

llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fd110b053f0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fd110b061a0>, root_client=<openai.OpenAI object at 0x7fd14069f6d0>, root_async_client=<openai.AsyncOpenAI object at 0x7fd110b05b70>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))